In [1]:
input_pdf_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\singular\TextualPage4.pdf" # Replace with your input PDF file path
output_pdf_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\singular\sample.pdf"  # Replace with your desired output PDF file path
file_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\files\SamcoFactSheet2024.pdf"

In [9]:
path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\TextualPdf.pdf"

#text

In [14]:
path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\TabularPdf.pdf"


In [64]:
import fitz 
import camelot
import statistics

In [1]:
import fitz  # PyMuPDF

def extract_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Usage
pdf_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\singular\TextualPage4.pdf"
text_data = extract_text(pdf_path)


In [ ]:
import re
text = text_data
def parse_text_to_dict(text):
    data_dict = {}
    lines = text.splitlines()
    current_header = None

    for line in lines:
        # Example pattern for headers (adjust as needed)
        header_match = re.match(r"^[A-Z][A-Za-z\s]+:$", line.strip())
        if header_match:
            current_header = header_match.group(0).strip(':')
            data_dict[current_header] = []
        elif current_header:
            data_dict[current_header].append(line.strip())

    return data_dict

# Usage
structured_data = parse_text_to_dict(text_data)


In [ ]:


def detect_columns(pdf_path, eps=50, min_samples=2):
    """
    Detect the number of text columns in each page of a PDF.

    Parameters:
    - pdf_path: str, path to the PDF file.
    - eps: float, maximum distance between two samples for them to be considered in the same neighborhood (DBSCAN parameter).
    - min_samples: int, number of samples in a neighborhood for a point to be considered a core point (DBSCAN parameter).

    Returns:
    - List of integers representing the number of columns detected on each page.
    """
    doc = fitz.open(pdf_path)
    column_counts = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text_dict = page.get_text("dict")

        # Collect x-coordinates of text blocks
        x_coords = []
        for block in text_dict.get("blocks", []):
            bbox = block.get("bbox", [])
            if bbox:
                x0 = bbox[0]  # x-coordinate of the left side of the block
                x_coords.append([x0])

        if x_coords:
            # Convert to numpy array
            x_coords = np.array(x_coords)

            # Apply DBSCAN clustering
            clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(x_coords)

            # Number of clusters (excluding noise if present)
            num_columns = len(set(clustering.labels_)) - (1 if -1 in clustering.labels_ else 0)
            column_counts.append(num_columns)
        else:
            column_counts.append(0)

    doc.close()
    return column_counts

# Example usage
pdf_path = file_path  # Replace with your PDF file path
columns_per_page = detect_columns(pdf_path)
for i, num_columns in enumerate(columns_per_page):
    print(f"Page {i + 1}: {num_columns} columns detected")


In [62]:

def annotate_pdf(input_path, output_path):
    # Open the PDF document
    pdf = fitz.open(input_path)
    
    # Iterate through each page
    for page_num in range(len(pdf)):
        page = pdf.load_page(page_num)
        # Extract text as a dictionary containing blocks, lines, and spans
        text_dict = page.get_text("dict")
        
        # Iterate through blocks
        for block in text_dict.get("blocks", []):
            # Check if the block contains lines
            if "lines" in block:
                # Iterate through lines
                for line in block["lines"]:
                    # Get the bounding box of the line
                    line_bbox = fitz.Rect(line["bbox"])
                    # Draw a rectangle around the line
                    page.draw_rect(line_bbox, color=(1, 0, 0), width=0.5)
    
    # Save the annotated PDF to the output path
    pdf.save(output_path)
    pdf.close()

# Example usage
input_pdf_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\singular\TextualPage4.pdf" # Replace with your input PDF file path
output_pdf_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\singular\sample.pdf"  # Replace with your desired output PDF file path
annotate_pdf(input_pdf_path, output_pdf_path)


In [56]:
def extract_pdf(path):
    
    pdf = fitz.open(path)
    page_content = []
    
    for pgn in range(len(pdf)):
        page = pdf.load_page(pgn)
        text = page.get_text("dict")
        page_content.append(text)
    return page_content
        
    # sort_by_y = []
    # for block in page_content:
    #     for lines in block['lines']:
    #         for span in lines['span']
    
    
content = extract_pdf(path)
print(f"\nPages:{len(content)}")


Pages:24


In [58]:
firstPage = content[3]

temp = []
text = []

for block in firstPage['blocks']:
    for lines in block['lines']:
        bbox = lines['bbox']
        line_text = "".join(span['text'] for span in lines['spans'])
        temp.append([bbox,line_text])
    


len(temp)

332

In [13]:
import pdfplumber

def extract_custom_tables_from_pdf(pdf_path):
# table_settings = {
#     "vertical_strategy": "lines",  # Options: "lines", "text", "mixed"
#     "horizontal_strategy": "lines",  # Options: "lines", "text", "mixed"
#     "snap_tolerance": 3,  # Distance tolerance for snapping lines
# }
    with pdfplumber.open(pdf_path) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            print(f"Extracting tables from page {page_number}...")
            tables = page.extract_tables()
            for table_number, table in enumerate(tables, start=1):
                print(f"Table {table_number}:")
                for row in table:
                    print(row)
                print("\n")

# Example usage
pdf_path =  r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\singular\TabularPage4.pdf"
extract_custom_tables_from_pdf(pdf_path)


Extracting tables from page 1...
